In [6]:
import pandas as pd
import numpy as np

In [7]:
master_df_final = pd.read_pickle('database\master_df_final_26Oct20.pickle')
nifty_200 = pd.read_csv('database\ind_nifty200list.csv')

In [8]:
master_df_final = master_df_final.merge(nifty_200.loc[:,['Symbol', 'Industry']], how='left', left_on=['Symbol'], right_on=['Symbol'])

In [9]:
master_df_final.loc[:,'1mForwardRet'] = master_df_final['1mreturn'].shift(1)

In [10]:
def using_nth(df):
    to_del = df.groupby('Symbol',as_index=False).nth(0)
    return pd.concat([df,to_del]).drop_duplicates(keep=False)

In [11]:
master_df_final = using_nth(master_df_final)

In [12]:
dates = master_df_final.loc[master_df_final.Symbol == 'ACC',['formatted_date']]
dates = dates[11:]
dates.reset_index(inplace=True)
dates = dates.formatted_date

In [15]:
factors_df = pd.DataFrame()
factors_list = ['3mreturn', 'Profitability', 'BookToPrice', 'Liquidity', 'LogMcap']
for i in range(len(dates)):
    current_date = dates[i]
#     print('Processing Date '+ str(current_date))
    current_df_final = master_df_final.loc[master_df_final.formatted_date == current_date,:].copy()
    current_df_final.loc[:,'MarketCapWeight'] = current_df_final.loc[:,'MarketCap'] / sum(current_df_final.loc[:,'MarketCap'])
    #current_df_final = current_df_final.merge(current_df_final.groupby(['Industry'])['MarketCap'].sum(), how='left', on=['Industry']).rename(columns={'MarketCap_y':'TotalMarketCap', 'MarketCap_x':'MarketCap'})
    #current_df_final.loc[:,'IndustryMarketCapWeight'] = current_df_final.loc[:,'MarketCap'] / current_df_final.loc[:,'TotalMarketCap']
    quintile_number = int(len(current_df_final.dropna())*0.20)
    current_fact_df = pd.DataFrame({'Date' : [current_date], 'Market': [sum(current_df_final['1mForwardRet'] * current_df_final['MarketCapWeight'])]})
    for factor in factors_list:
#         print('Processing '+factor)
        current_fact_df.loc[:,factor] = np.mean(current_df_final.sort_values([factor], ascending=[0]).tail(quintile_number)['1mForwardRet']) - np.mean(current_df_final.sort_values([factor], ascending=[0]).head(quintile_number)['1mForwardRet'])
    #for industry in industry_factors:
    #    print('Processing '+industry)
    #    current_fact_df.loc[:,industry] = sum(current_df_final.loc[current_df_final.Industry == current_industry,:]['IndustryMarketCapWeight'] * current_df_final.loc[current_df_final.Industry == current_industry,:]['1mForwardRet'])
    factors_df = pd.concat([factors_df, current_fact_df], axis=0)

In [18]:
factors_df.to_pickle('database\monthly_factor_returns_26Oct20.pickle')